# jPCA_base.ipynb

This notebook has the code necessary to implement the jPCA algorithm, as described in:
https://static-content.springer.com/esm/art%3A10.1038%2Fnature11129/MediaObjects/41586_2012_BFnature11129_MOESM225_ESM.pdf

We have $ct$ sample points of a time-variant vector signal $X(t) = [x_1(t), \dots, x_n(t)]$. These sample points are in the matrix $X \in \Re^{ct \times n}$.  
We want to find the skew-symmetric matrix $M in \Re^{n \times n}$ that best fits $\dot{X}(t) = X(t) M$.  
In other words, $M$ minimizes $\| \dot{X} - X M \|^2$ with the Frobenius norm.

Since $M$ is skew-symmetric, it will have $n(n-1)/2$ independent entries, which we arrange into the column vector $k$, and solve the equivalent problem using "unrolled" columns. This is expressed as:  
$k^* = \text{armgin}_{k \in \Re^{n(n-1)/2}} \| \dot{x} - \tilde{X} H k \|_2$,  
where $\tilde{X} \in \Re^{(ct \ n) \times n^2}$ is a block matrix with $n$ copies 
of $X$ in its main diagonal, 
$H \in \Re^{n^2 \times n(n-1)/2}$ is a linear transformation that puts the elements of $k$ into the unrolled version of $M$, and $\dot{x}$ is the unrolled version of $\dot{X}$.

The solution to this problem is $k^* = (\tilde{X} H) \backslash \dot{x}$

In [61]:
import numpy as np
import scipy as sp
from scipy.integrate import solve_ivp

In [52]:
# Create the data matrix X
# Each row of corresponds to a different time point

# Creating X by evolving a dynamical system
n = 6 # dimensionality of the data
ct = 40 # number of time points
# Using a linear skew-symmetric matrix
mat = np.random.rand(n,n)-0.5
Msk = mat - mat.transpose()

def dt_fun(t, y):
    return np.matmul(Msk, y)

t0 = 0. # initial time of integration
tf = 2. # final time of integration
X0 = 2.*(np.random.rand(n)-0.5) # initial state
t_points = np.linspace(t0, tf, ct)
sol = solve_ivp(dt_fun, [t0,tf], X0, t_eval=t_points)
X = sol.y.transpose()

# standardize the X data
X = X - np.mean(X)
X = X / np.std(X)

# create the block-matrix version of X
Xtilde = sp.linalg.block_diag(*([X]*n))

In [54]:
# Create the H matrix
n = X.shape[1]
ct = X.shape[0]
L = np.zeros((n,n), dtype=int)
c = 0
for row in range(n):
    for col in range(row+1, n):
        L[row, col] = c
        L[col, row] = c
        c += 1
        
H = np.zeros((n*n, int(0.5*n*(n-1))))
for col in range(n):
    for row in range(n):
        if col > row:
            H[n*col+row, L[col,row]] = 1.
        elif row > col:
            H[n*col+row, L[col,row]] = -1.

In [55]:
# Approximate the derivatives of X
Xp = np.zeros_like(X)
t_bit = t_points[1] - t_points[0]
Xp[1:,:] = (X[1:,:] - X[:-1,:]) / t_bit

# prev_row = np.zeros(n)
# for row in range(1, ct):
#     Xp[row-1,:] = (X[row,:] - X[row-1,:]) / t_bit
# Xp[-1,:] = Xp[-2,:]

xp = Xp.flatten('F')

In [56]:
# calculate the solution directly from the formula
kstar = np.matmul(np.linalg.pinv(np.matmul(Xtilde, H)), xp)

# reconstruct the matrix Msk that generated the data
Mstar = np.matmul(H, kstar).reshape(n,n)
print(Mstar)
print(Msk)

[[ 0.         -0.35340092  0.52222587 -0.10543001  0.11408025 -0.24255443]
 [ 0.35340092  0.          0.542875   -1.02418614  0.43698895 -0.21896857]
 [-0.52222587 -0.542875    0.         -0.66303951 -0.40146722  0.19494389]
 [ 0.10543001  1.02418614  0.66303951  0.         -0.24335933  0.55007975]
 [-0.11408025 -0.43698895  0.40146722  0.24335933  0.          0.57280916]
 [ 0.24255443  0.21896857 -0.19494389 -0.55007975 -0.57280916  0.        ]]
[[ 0.         -0.18470449  0.87605911 -0.24233307 -0.47498955 -0.34227083]
 [ 0.18470449  0.          0.16014874 -0.91875513  0.34569415 -0.17027741]
 [-0.87605911 -0.16014874  0.         -0.25820942 -0.12366712 -0.36612742]
 [ 0.24233307  0.91875513  0.25820942  0.          0.0100411   0.49904048]
 [ 0.47498955 -0.34569415  0.12366712 -0.0100411   0.          0.30846109]
 [ 0.34227083  0.17027741  0.36612742 -0.49904048 -0.30846109  0.        ]]


In [57]:
# Frobenius norm of the matrix difference
# Mstar should be the transposed????
norm1 = np.linalg.norm(Mstar- Msk)
print(norm1)

# Angle between Mstar and Msk
angM= np.arccos( (Mstar*Msk).sum() / (np.linalg.norm(Mstar)*np.linalg.norm(Msk)))
print(angM)

# Frobenius norm of derivatives matrix minus its reconstruction
Xp_rec = np.matmul(X, Mstar)
norm2 = np.linalg.norm(Xp - Xp_rec)
print(norm2)

# Angle between Xp and its reconstruction
ang = np.arccos( (Xp * Xp_rec).sum() / (np.linalg.norm(Xp)*np.linalg.norm(Xp_rec)) )
print(ang)

1.667455619112301
0.6783248931499707
37.43646631939747
2.6566131505044894


In [58]:
# print H and the auxiliary matrices
print("H =")
print(H, end='\n\n')
print("L =")
print(L, end='\n\n')

k = np.arange(1,int(0.5*n*(n-1)+1))
print("k =")
print(k, end='\n\n')
M = np.matmul(H,k)
print("M =")
print(M, end='\n\n')

H =
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  

In [62]:
# 2) Coefficients of determination
# 2.1) Obtain unconstrained M matrix
M_uncons = np.matmul(np.linalg.pinv(X), Xp)
# 2.2) Reconstruct Xp with M_uncons
Xp_uncons = np.matmul(X, M_uncons)
# 2.3) Reconstruct Xp with Mstar
Xp_skew = np.matmul(X, Mstar)
# 2.4) Calculate residual sums of squares
SSres_uncons = ((Xp - Xp_uncons) * (Xp - Xp_uncons)).sum()
SSres_skew = ((Xp - Xp_skew) * (Xp - Xp_skew)).sum()
# 2.5) Calculate the total sum of squares
SStot = ((Xp-Xp.mean())*(Xp-Xp.mean())).sum()
# 2.6) Calculate coefficients of determination
R2_uncons = 1. - (SSres_uncons / SStot)
R2_skew = 1. - (SSres_skew / SStot)

print("R2 unconstrained: %f" % (R2_uncons))
print("R2 skew symmetric: %f" % (R2_skew))

R2 unconstrained: 0.988177
R2 skew symmetric: -2.840164


In [60]:
a = np.array([[0,1],[-1,0]])
eig_vals, eig_vecs = np.linalg.eig(a)
print(a)
print("Eigenvalues: ")
print(eig_vals)
print("Eigenvectors: ")
print(eig_vecs)

[[ 0  1]
 [-1  0]]
Eigenvalues: 
[0.+1.j 0.-1.j]
Eigenvectors: 
[[0.70710678+0.j         0.70710678-0.j        ]
 [0.        +0.70710678j 0.        -0.70710678j]]
